In [134]:
# Import packages
import os
import warnings
import cartopy.crs as ccrs
from cartopy import feature as cf
import matplotlib.pyplot as plt
import numpy.ma as ma
import xarray as xr
import rioxarray as rxr
import glob as glob
import pandas as pd
import numpy as np
warnings.simplefilter('ignore')

In [ ]:
## Code for preprossesing of daily data files of .hdf format of MODIS satellite

In [135]:
years = [str(i) for i in range(2001,2021)]                                  # string format
years_int = [int(i) for i in range(2001,2021)]                              # integer format
day=[str(i).zfill(3) for i in range(1,367)]                                 # string list with numbers from 001 to 366
start_date = '2001-01-01'                                                   # start date (To be modified according to your data files)
end_date = '2020-12-31'                                                     # end date (To be modified according to your data files)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
timeseries=pd.DataFrame(date_range)
cnt=timeseries.count()
q=0

compiled=[None]*20                                                          # list length to be changed according to your no. of years of data
for y in range(0,20):                                                       # loop closing number same as no. of years of data
    
    # To get year (integer input) from the user
    # year = int(input("Enter a year: "))
    
    # divided by 100 means century year (ending with 00)
    # century year divided by 400 is leap year
    if (years_int[y] % 400 == 0) and (years_int[y] % 100 == 0):
        LST=[None]*366
        print(years[y])
        # print('This is a leap year')
        for d in range(0,366):
            try:
                file=glob.glob("<PATH>/MOD11C1.A"+years[y]+day[d]+"*.hdf")  # Change the path according to your files
                LS = rxr.open_rasterio(file[0], masked=True)
                LS = LS["LST_Day_CMG"].squeeze().sel(x=slice('72.75','73.00'), y=slice('19.35','18.85'))*0.02   # variable and lat, lon slicing can be changed accordingly
                LS = LS.rename({'x':'lon'}).rename({'y':'lat'})
                LS = LS.reset_coords('band', drop=True).reset_coords('spatial_ref', drop=True)
                LST[d] = LS.assign_coords({'time':date_range[q]}).expand_dims('time').to_dataset()
                q=q+1
            except:
                LST[d]=xr.full_like(LST[0],np.nan)                          # Missing daily files data filled with Nan value datasets
                var=LST[d].drop_dims('time').assign_coords({'time':date_range[q]}).expand_dims('time')
                LST[d]['time']=var['time']
                q=q+1
                continue
        compiled[y]=LST
                
    # not divided by 100 means not a century year
    # year divided by 4 is a leap year
    elif (years_int[y] % 4 ==0) and (years_int[y] % 100 != 0):
        LST=[None]*366
        print(years[y])
        # print('This is a leap year')
        for d in range(0,366):
            try:
                file=glob.glob("<PATH>/MOD11C1.A"+years[y]+day[d]+"*.hdf")
                LS = rxr.open_rasterio(file[0], masked=True)
                LS = LS["LST_Day_CMG"].squeeze().sel(x=slice('72.75','73.00'), y=slice('19.35','18.85'))*0.02
                LS = LS.rename({'x':'lon'}).rename({'y':'lat'})
                LS = LS.reset_coords('band', drop=True).reset_coords('spatial_ref', drop=True)
                LST[d] = LS.assign_coords({'time':date_range[q]}).expand_dims('time').to_dataset()
                q=q+1
            except:
                LST[d]=xr.full_like(LST[0],np.nan)
                var=LST[d].drop_dims('time').assign_coords({'time':date_range[q]}).expand_dims('time')
                LST[d]['time']=var['time']
                q=q+1
                continue
        compiled[y]=LST
    
    # if not divided by both 400 (century year) and 4 (not century year)
    # year is not leap year
    else:
        LST=[None]*365
        print(years[y])
        for d in range(0,365):
            try:
                file=glob.glob("<PATH>/MOD11C1.A"+years[y]+day[d]+"*.hdf")
                LS = rxr.open_rasterio(file[0], masked=True)
                LS = LS["LST_Day_CMG"].squeeze().sel(x=slice('72.75','73.00'), y=slice('19.35','18.85'))*0.02
                LS = LS.rename({'x':'lon'}).rename({'y':'lat'})
                LS = LS.reset_coords('band', drop=True).reset_coords('spatial_ref', drop=True)
                LST[d] = LS.assign_coords({'time':date_range[q]}).expand_dims('time').to_dataset()
                q=q+1
            except:
                LST[d]=xr.full_like(LST[0],np.nan)
                var=LST[d].drop_dims('time').assign_coords({'time':date_range[q]}).expand_dims('time')
                LST[d]['time']=var['time']
                q=q+1
                continue
        compiled[y]=LST

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [136]:
year_concat=[None]*20
for i in range(0,20):
    year_concat[i] = xr.concat(compiled[i], dim='time')

In [137]:
final_datas=xr.concat(year_concat,dim='time')

In [138]:
final_datas.to_netcdf('<PATH>/All_Mumbai_MODIS_LST_Day_CMG.nc', mode='w')